In [17]:
import scanpy as sc
import os 
import numpy as np
import pandas as pd
import time as tm
from sklearn.svm import LinearSVC
import scanpy as sc
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report
from sklearn.calibration import CalibratedClassifierCV

In [26]:
adata = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/scgpt_dataset/demo_train.h5ad")

In [27]:
adata

AnnData object with n_obs × n_vars = 10600 × 3000
    obs: 'Celltype', 'celltype'
    var: 'Gene Symbol'

In [28]:
eplison = 10
np.random.seed(2023)
rand_data = np.asarray(np.random.rand(adata.X.shape[0], adata.X.shape[1]))

In [29]:
X_copy = adata.X.toarray()

In [30]:
X_posioned = X_copy + eplison * rand_data

In [31]:
adata.layers['poi'] = X_posioned 

In [34]:
sc.tl.rank_genes_groups(adata, groupby='celltype', n_genes=40, method='wilcoxon')

In [40]:
emp_lit = []

for i in adata.obs['celltype'].unique():
    emp_lit += list(adata.uns['rank_genes_groups']['names'][i])

In [41]:
adata = adata[:, sorted(set(emp_lit))]

In [42]:
adata

View of AnnData object with n_obs × n_vars = 10600 × 400
    obs: 'Celltype', 'celltype'
    var: 'Gene Symbol'
    uns: 'rank_genes_groups'
    layers: 'poi'

In [52]:
X_pos_train, X_pos_test, y_pos_train, y_pos_test =sklearn.model_selection.train_test_split(adata.layers['poi'], adata.obs['celltype'] , test_size=0.33, random_state=2023)

In [54]:
X_pos_test.shape

(3498, 400)

In [48]:
# adata.X.toarray().shape

In [44]:
X_train = adata.X.toarray()
y_train = adata.obs['celltype'].values

In [45]:
Classifier = LinearSVC()
clf = CalibratedClassifierCV(Classifier)
clf.fit(X_train, y_train)

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/gpfs/gibbs/projec

CalibratedClassifierCV(estimator=LinearSVC())

In [55]:
predicted_y = clf.predict(X_pos_test)
print(classification_report(y_pos_test, predicted_y, digits=4))

              precision    recall  f1-score   support

          PP     0.3846    0.0420    0.0758       119
         PSC     0.7083    0.1006    0.1762       169
      acinar     0.5000    0.0026    0.0053       378
       alpha     1.0000    0.0135    0.0267      1036
        beta     0.0000    0.0000    0.0000      1004
       delta     0.6111    0.0425    0.0794       259
      ductal     0.5517    0.0780    0.1368       410
 endothelial     0.0437    1.0000    0.0838        88
     epsilon     0.0045    0.8571    0.0090         7
  macrophage     0.3571    0.3571    0.3571        14
        mast     0.0000    0.0000    0.0000         5
     schwann     0.3333    0.2000    0.2500         5
      t_cell     0.0000    0.0000    0.0000         4

    accuracy                         0.0515      3498
   macro avg     0.3457    0.2072    0.0923      3498
weighted avg     0.5104    0.0515    0.0454      3498



/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [56]:
predicted_y = clf.predict(adata.layers['poi'])
print(classification_report(adata.obs['celltype'], predicted_y, digits=4))

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          PP     0.3226    0.0281    0.0517       356
         PSC     0.7188    0.0878    0.1565       524
      acinar     0.8235    0.0122    0.0241      1144
       alpha     1.0000    0.0143    0.0283      3136
        beta     0.0000    0.0000    0.0000      2966
       delta     0.5962    0.0391    0.0734       793
      ductal     0.5753    0.0829    0.1450      1290
 endothelial     0.0448    0.9927    0.0857       273
     epsilon     0.0049    0.9524    0.0098        21
  macrophage     0.3214    0.3462    0.3333        52
        mast     1.0000    0.0400    0.0769        25
     schwann     0.1429    0.0769    0.1000        13
      t_cell     0.0385    0.2857    0.0678         7

    accuracy                         0.0534     10600
   macro avg     0.4299    0.2276    0.0887     10600
weighted avg     0.5510    0.0534    0.0478     10600



/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
import scanpy as sc
import os 
import numpy as np
import pandas as pd
import time as tm
from sklearn.svm import LinearSVC
import scanpy as sc
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report
from sklearn.calibration import CalibratedClassifierCV

In [37]:
adata = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/scgpt_dataset/demo_train.h5ad")

In [38]:
adata

AnnData object with n_obs × n_vars = 10600 × 3000
    obs: 'Celltype', 'celltype'
    var: 'Gene Symbol'

In [39]:
eplison = 10
np.random.seed(2023)
rand_data = np.asarray(np.random.rand(adata.X.shape[0], adata.X.shape[1]))

In [40]:
X_copy = adata.X.toarray()

In [41]:
X_posioned = X_copy + eplison * rand_data

In [42]:
adata.layers['poi'] = X_posioned 

In [43]:
sc.tl.rank_genes_groups(adata, groupby='celltype', n_genes=40, method='wilcoxon')
emp_lit = []
for i in adata.obs['celltype'].unique():
    emp_lit += list(adata.uns['rank_genes_groups']['names'][i])
adata = adata[:, sorted(set(emp_lit))]

In [44]:
X_pos_train, X_pos_test, y_pos_train, y_pos_test =sklearn.model_selection.train_test_split(adata.layers['poi'], adata.obs['celltype'] , test_size=0.33, random_state=2023)

In [59]:
X_train = adata.X.toarray()
y_train = adata.obs['celltype'].values

In [60]:
import torch
import torch.nn.functional as F
import lightning as L
import os
import pandas as pd
import numpy as np
import pickle
import sklearn.model_selection
import scipy.stats
import sklearn.metrics
import scanpy as sc

from torch.utils.data import DataLoader
from torch import nn
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch.callbacks.early_stopping import EarlyStopping


In [61]:
layers =[adata.X.shape[1], 512, len(set(adata.obs['Celltype']))]

In [62]:
# layers

In [63]:
class Encoder(nn.Module):
    def __init__(self, k=3):
        super().__init__()
        self.shared_layers = nn.Sequential(nn.Linear(layers[0], layers[1]), 
                                nn.BatchNorm1d(layers[1]),
                                nn.ReLU(), 
                                nn.Dropout(input_dropout),
                                nn.Linear(layers[1], layers[1]),
                                nn.BatchNorm1d(layers[1]),
                                nn.ReLU(), 
                                nn.Dropout(input_dropout),
                                nn.Linear(layers[1], layers[2])
                               )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        
        return self.softmax (self.shared_layers(x))
        



class LitAutoEncoder(L.LightningModule):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder
        self.loss = nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        
        output_label = self.encoder(x)
        
        loss = self.loss(output_label, y.long())
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, y = batch
        
        output_label = self.encoder(x)
        
        val_loss = self.loss(output_label, y.long())
        
        self.log("val_loss", val_loss.item())
        return val_loss

        
    def test_step(self, batch, batch_idx):
        # this is the test loop
        x, y = batch
        
        output_label = self.encoder(x)
        
        test_loss = self.loss(output_label, y.long())
        self.log("test_loss", test_loss.item())
        return test_loss
        
    def forward(self, x):
        return self.encoder(x)

    def configure_optimizers(self):
            optimizer = torch.optim.Adam(
                params=self.parameters(), 
                lr=eta
)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer,
                patience=10,
                verbose=True
            )
            return {
               'optimizer': optimizer,
               'lr_scheduler': scheduler, # Changed scheduler to lr_scheduler
               'monitor': 'val_loss'
           }


In [64]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [65]:
label_data = le.fit_transform(y_train)

In [66]:
adata.X

<10600x400 sparse matrix of type '<class 'numpy.float32'>'
	with 1085044 stored elements in Compressed Sparse Row format>

In [67]:
import sklearn.model_selection 

In [68]:
X_tr, X_val, y_tr, y_val = sklearn.model_selection.train_test_split(X_train,label_data , random_state=2023)

In [69]:
X_test = X_pos_test
y_test = le.transform(y_pos_test)

In [70]:
X_tr, X_val, X_train, X_test, y_tr, y_val, y_train, y_test =torch.FloatTensor(X_tr),torch.FloatTensor(X_val),torch.FloatTensor(X_tr),torch.FloatTensor(X_test),torch.FloatTensor(y_tr), torch.FloatTensor(y_val), torch.FloatTensor(y_tr), torch.FloatTensor(y_test)


train_dataset = torch.utils.data.TensorDataset(X_tr, y_tr)
valid_dataset = torch.utils.data.TensorDataset(X_val, y_val)
test_dataset = torch.utils.data.TensorDataset(X_tr, y_tr)

epochs = 1000 
act_func = 'GELU'
dropout = 0.5 
input_dropout = 0.2
eta = 1e-4
norm = 'tanh' 
drug_num_dim = 16

model = LitAutoEncoder(Encoder())

print(model.log) #see log and log dir for model place

lr_monitor = LearningRateMonitor(logging_interval='step')

train_loader = DataLoader(train_dataset, batch_size=4096, num_workers=5, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=2048, num_workers=5)

# train with both splits
trainer = L.Trainer(callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=100)], max_epochs=1000)

trainer.log_dir

trainer.fit(model, train_loader, valid_loader)

trainer.test(model, dataloaders=DataLoader(test_dataset, batch_size=1024, num_workers=5))

# checkpoint = torch.load("/gpfs/gibbs/pi/zhao/tl688/adversial_training/lightning_logs/version_24415172/checkpoints/epoch=399-step=800.ckpt")
# model.load_state_dict(checkpoint["state_dict"])


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.8 /gpfs/gibbs/project/zhao/tl688/conda_envs/deeprob ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

<bound method LightningModule.log of LitAutoEncoder(
  (encoder): Encoder(
    (shared_layers): Sequential(
      (0): Linear(in_features=400, out_features=512, bias=True)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.2, inplace=False)
      (8): Linear(in_features=512, out_features=13, bias=True)
    )
    (softmax): Softmax(dim=1)
  )
  (loss): CrossEntropyLoss()
)>


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     1.695825219154358     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.695825219154358}]

In [71]:
for m in model.encoder.modules():

    for child in m.children():

        if type(child) == nn.BatchNorm1d:

            child.track_running_stats = False

            child.running_mean = None

            child.running_var = None
model.encoder.eval()

Encoder(
  (shared_layers): Sequential(
    (0): Linear(in_features=400, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=512, out_features=13, bias=True)
  )
  (softmax): Softmax(dim=1)
)

In [72]:
X_test = torch.FloatTensor(X_pos_test)
y_test = torch.FloatTensor(le.transform(y_pos_test))

In [73]:
with torch.no_grad():
    y_pred = model.encoder(X_test)
    _ , y_pred = torch.max(y_pred, 1)

In [74]:
y_pred

tensor([0, 3, 4,  ..., 2, 6, 3])

In [75]:
y_test

tensor([4., 3., 4.,  ..., 6., 3., 3.])

In [76]:
sklearn.metrics.classification_report(y_test, y_pred)

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

'              precision    recall  f1-score   support\n\n         0.0       0.73      0.38      0.50       119\n         1.0       0.81      0.67      0.74       169\n         2.0       0.73      0.64      0.68       378\n         3.0       0.83      0.86      0.85      1036\n         4.0       0.78      0.82      0.80      1004\n         5.0       0.64      0.49      0.55       259\n         6.0       0.56      0.77      0.65       410\n         7.0       0.88      0.69      0.78        88\n         8.0       0.00      0.00      0.00         7\n         9.0       1.00      0.43      0.60        14\n        10.0       0.00      0.00      0.00         5\n        11.0       0.00      0.00      0.00         5\n        12.0       0.00      0.00      0.00         4\n\n    accuracy                           0.75      3498\n   macro avg       0.54      0.44      0.47      3498\nweighted avg       0.75      0.75      0.75      3498\n'